In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/NLP_POS/superai2_pos_tagging.zip

In [ ]:
path = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/train/"

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install tqdm

In [ ]:
!pip install keras-crf

In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import gc
import tensorflow as tf 
import numpy as np
from keras import layers
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from tqdm import tqdm
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_crf import CRFModel

In [ ]:
dir_list = os.listdir(path)

In [ ]:
dir_list.sort()
dir_list=dir_list[1:]

In [ ]:
word=[]
pos=[]
for file in dir_list:
  file2 = open(path+file, 'r')
  Lines = file2.readlines()
  for line in Lines:
    if line == "\n":
      word.append("_")
      pos.append("PU")
    else:
      word.append(line.split("\t")[0])
      pos.append(line.split("\t")[1])
  file2.close()

In [ ]:
All_data = pd.DataFrame()
All_data["word"]=word
All_data["POS"]=pos

In [ ]:
del dir_list,word,pos,Lines 
gc.collect()

275

In [ ]:
All_data

,word,POS
0,สุรยุทธ์,NN
1,ยัน,VV
2,ปฏิเสธ,VV
3,ลงนาม,VV
4,_,PU
...,...,...
2778031,ถอน,VV
2778032,รายชื่อ,NN
2778033,ด้วย,PS
2778034,.,PU


In [ ]:
model_emb = SentenceTransformer('mrp/simcse-model-roberta-base-thai')

In [ ]:
n=len(All_data)//256

In [ ]:
emb_list=[]
for i in tqdm(range(n+1)):
  if i==n:
    embeddings = model_emb.encode(list(All_data.iloc[256*i:len(All_data["word"]),0]))
  else:
    embeddings = model_emb.encode(list(All_data.iloc[256*i:(256*(i+1)),0]))  

  emb_list=emb_list+list(embeddings)

100%|██████████| 10852/10852 [29:10<00:00,  6.20it/s]


In [ ]:
del embeddings
gc.collect()

134

In [ ]:
len(emb_list[100])

768

In [ ]:
Pos_unique = All_data['POS'].unique()

In [ ]:
Pos_unique=list(Pos_unique)

In [ ]:
def converlabel(Pos_unique,s):
  ind = Pos_unique.index(s)
  return ind

In [ ]:
relabel=[]
for i in All_data['POS']:
  relabel.append(converlabel(Pos_unique,i))

In [ ]:
new_emb_list = []
print(len(emb_list))
for i in tqdm(range(0,len(emb_list),16)):
  if i in list(range(16)): 
    temp0 = [0]*768
    temp = emb_list[0:i+1]
    for j in range(16-i-1):
      temp.insert(0,temp0)
  elif i>=16 and i < len(emb_list)-16: 
    temp = emb_list[i-8:i+8]
  else:
    temp = emb_list[i:len(emb_list)]
    temp0 = [0]*768
    for j in range(16-(len(emb_list)-i)):
      temp.append(temp0)
  new_emb_list.append(temp)

2778036


100%|██████████| 173628/173628 [00:00<00:00, 221378.27it/s]


In [ ]:
del emb_list
gc.collect()

129

In [ ]:
del All_data,temp,temp0
gc.collect()

88

In [ ]:
gc.collect()
new_emb_list = np.asarray(new_emb_list,dtype='float32')

In [ ]:
new_emb_list.shape

(173628, 16, 768)

In [ ]:
def text_clasifier_lstm(input_shape):

    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    input = Input(shape=input_shape)  

    X = Bidirectional(layers.LSTM(256, return_sequences=True,dropout=0.1))(input)
    X = Bidirectional(layers.LSTM(256, activation='relu', return_sequences=True,go_backwards=True,dropout=0.1))(X)
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(X)
    # Add dropout with a probability of 0.1
    X = Dropout(0.3)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128,return_sequences=True)(X)
    # Add dropout with a probability of 0.1
    X = Dropout(0.3)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(64)(X)
    # Add a softmax activation
    #X = Activation('softmax')(X)
    #main_lstm = layers.TimeDistributed(Dense(16, activation="relu"))(X)
    base=tf.keras.Model(inputs=input,outputs=X)
    model=CRFModel(base,16)
    # CRF
    #crf = CRF(17)  # CRF layer
    #out = crf(main_lstm)  # output
    # Create Model instance which converts sentence_indices into X.
    #model = Model(input, out)
    
    return model

In [ ]:
model = text_clasifier_lstm((16,768))

In [ ]:
model.summary()

Model: "crf_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 768)]    0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 16, 512)      2099200     ['input_1[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 16, 512)     1574912     ['bidirectional[0][0]']          
 )                                                                                                
                                                                                                  
 lstm_2 (LSTM)                  (None, 16, 128)      328192      ['bidirectional_1[0][0]']

In [ ]:
Y_train = []
for i in tqdm(range(0,len(relabel),16)):
  if i in list(range(16)): 
    temp0 = 2
    temp = relabel[0:i+1]
    for j in range(16-i-1):
      temp.insert(0,temp0)
  elif i>=10 and i < len(relabel)-16: 
    temp = relabel[i-8:i+8]
  else:
    temp = relabel[i:len(relabel)]
    temp0 = 2
    for j in range(16-(len(relabel)-i)):
      temp.append(temp0)
  Y_train.append(temp)

100%|██████████| 173628/173628 [00:00<00:00, 289026.89it/s]


In [ ]:
del temp,temp0,relabel
gc.collect()
Y_train = np.asarray(Y_train,dtype='float32')

In [ ]:
Y_train.shape

(173628, 16)

In [ ]:
path1 = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/eval/"

In [ ]:
dir_list1 = os.listdir(path1)
dir_list1.sort()
dir_list1=dir_list1[1:]

In [ ]:
word1=[]
pos1=[]
for file in dir_list1:
  file3 = open(path1+file, 'r')
  Lines = file3.readlines()
  for line in Lines:
    if line == "\n":
      word1.append("_")
      pos1.append("PU")
    else:
      word1.append(line.split("\t")[0])
      pos1.append(line.split("\t")[1])
  file3.close()

In [ ]:
All_data1 = pd.DataFrame()
All_data1["word"]=word1
All_data1["POS"]=pos1

In [ ]:
All_data1

,word,POS
0,สลด,VV
1,ผัว,NN
2,เมีย,NN
3,เฒ่า,NN
4,ถูก,AX
...,...,...
246475,ขอ,VV
246476,ไว้,AV
246477,ดูแล,VV
246478,เอง,AV


In [ ]:
del dir_list1, word1,pos1
gc.collect()

0

In [ ]:
m=len(All_data1)//256

In [ ]:
emb_list1=[]
for i in tqdm(range(m+1)):
  if i==m:
    embeddings = model_emb.encode(list(All_data1.iloc[256*i:len(All_data1["word"]),0]))
  else:
    embeddings = model_emb.encode(list(All_data1.iloc[256*i:(256*(i+1)),0]))
  emb_list1=emb_list1+list(embeddings)

100%|██████████| 963/963 [01:58<00:00,  8.11it/s]


In [ ]:
len(emb_list1)

246480

In [ ]:
relabel1=[]
for i in All_data1['POS']:
  relabel1.append(converlabel(Pos_unique,i))

In [ ]:
len(relabel1)

246480

In [ ]:
del embeddings
gc.collect()

576

In [ ]:
new_emb_list1 = []
for i in tqdm(range(0,len(emb_list1),16)):
  if len(emb_list1)-i<16:
    temp = emb_list1[i:len(emb_list1)]
    temp0 = [0]*768
    for j in range(16-(len(emb_list1)-i)):
      temp.append(temp0)
    new_emb_list1.append(temp)
  else:
    new_emb_list1.append(emb_list1[i:i+16])

100%|██████████| 15405/15405 [00:00<00:00, 626528.46it/s]


In [ ]:
#del temp,temp0,embeddings,emb_list1
#gc.collect()
new_emb_list1 = np.asarray(new_emb_list1,dtype='float32')

In [ ]:
new_emb_list1.shape

(15405, 16, 768)

In [ ]:
Y_val = []
for i in tqdm(range(0,len(relabel1),16)):
  if len(emb_list1)-i<16:
    temp = relabel1[i:len(relabel1)]
    temp0=0
    for j in range(16-(len(relabel1)-i)):
      temp.append(temp0)
    Y_val.append(temp)
  else:
    Y_val.append(relabel1[i:i+16])

100%|██████████| 15405/15405 [00:00<00:00, 425720.17it/s]


In [ ]:
Y_val=np.asarray(Y_val,dtype='float32')

In [ ]:
Y_val.shape

(15405, 16)

In [ ]:
gc.collect()

NameError: ignored

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/NLP_POS/best_model2.h5')

adam = tf.keras.optimizers.Adam(lr=0.0005)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(new_emb_list, Y_train, epochs = 5, batch_size = 32, shuffle=True,validation_data=(new_emb_list1,Y_val) ) 

Epoch 1/3
5426/5426 [==============================] - 683s 123ms/step - decode_sequence_accuracy: 0.7490 - loss: 11.1309 - val_decode_sequence_accuracy: 0.9325 - val_loss: 5.9282
Epoch 2/3
5426/5426 [==============================] - 652s 120ms/step - decode_sequence_accuracy: 0.9313 - loss: 3.3301 - val_decode_sequence_accuracy: 0.9491 - val_loss: 4.6642
Epoch 3/3
 592/5426 [==>...........................] - ETA: 9:32 - decode_sequence_accuracy: 0.9441 - loss: 2.7442

In [ ]:
model.save('/content/drive/MyDrive/POSTAG/model1.h5')

In [ ]:
?;'path2 = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/test/"

In [ ]:
dir_list2 = os.listdir(path2)
dir_list2.sort()
dir_list2=dir_list2[1:]

In [ ]:
All_data2=pd.DataFrame()
for file in dir_list2:
  data = pd.DataFrame(pd.read_csv(path2+file,sep='\t',header=None,names=['word','POS','NER','boundary'], quotechar=' '))
  All_data2=All_data2.append(data, ignore_index=True)

In [ ]:
All_data2

,word,POS,NER,boundary
0,จีน,NN,B_LOC,B_CLS
1,-,PU,O,I_CLS
2,อินเดีย,NN,B_LOC,I_CLS
3,เสี่ยง,VV,O,I_CLS
4,สูญเสีย,VV,O,I_CLS
...,...,...,...,...
207273,ครหา,VV,O,I_CLS
207274,เกี่ยวกับ,VV,O,I_CLS
207275,ความ,FX,O,I_CLS
207276,ไม่,NG,O,I_CLS


In [ ]:
X_test_indices = sentences_to_indices(All_data2['word'], word2vec, 12)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
relabel2=[]
for i in All_data2['POS']:
  relabel2.append(converlabel(Pos_unique,i))

In [ ]:
Y_test_oh = convert_to_one_hot(relabel2, 16)

In [ ]:
solution = model.predict(X_test_indices)

In [ ]:
def inverselabel(Pos_unique,x):
  return Pos_unique[x]

In [ ]:
result=[]
for i in solution:
  result.append(inverselabel(Pos_unique,np.argmax(i)))

In [ ]:
All_data2["result"]=result

In [ ]:
All_data2

,word,POS,NER,boundary,result
0,จีน,NN,B_LOC,B_CLS,NN
1,-,PU,O,I_CLS,PU
2,อินเดีย,NN,B_LOC,I_CLS,NN
3,เสี่ยง,VV,O,I_CLS,VV
4,สูญเสีย,VV,O,I_CLS,VV
...,...,...,...,...,...
207273,ครหา,VV,O,I_CLS,VV
207274,เกี่ยวกับ,VV,O,I_CLS,VV
207275,ความ,FX,O,I_CLS,FX
207276,ไม่,NG,O,I_CLS,NG


In [ ]:
path3 = "/content/drive/MyDrive/NLP_POS/"

In [ ]:
!unzip "/content/drive/MyDrive/NLP_POS/pos_test.txt.zip"

unzip:  cannot find or open /content/drive/MyDrive/NLP_POS/pos_test.txt.zip, /content/drive/MyDrive/NLP_POS/pos_test.txt.zip.zip or /content/drive/MyDrive/NLP_POS/pos_test.txt.zip.ZIP.


In [ ]:
file1 = open('/content/super-ai-engineer-2021-nlp-pos-tagging/pos_test.txt', 'r')
Lines = file1.readlines()

In [ ]:
data_t=[]
for line in Lines:
  data_t.append(line)

In [ ]:
dat=[]
for i in data_t:
  dat.append(i.rstrip("\n"))

In [ ]:
Data3=pd.DataFrame(dat)

In [ ]:
result3=[]
for i,row in Data3.iterrow():
  if row[0]==' ':
    result3.append(' ')
  else : 
    All_data3['word'], word2vec, 12)
    result3.append(X_test1_indices = sentences_to_indices(All_data3['word'], word2vec, 12))

In [ ]:
All_data4 = pd.DataFrame(pd.read_csv("/content/super-ai-engineer-2021-nlp-pos-tagging/pos_sample_submission.csv",sep='\t',header=None,names=['word']))

In [ ]:
Data3

,0
0,อย่างไรก็ตาม
1,เครื่อง
2,บิน
3,แอร์บัส
4,
...,...
103909,พร้อม
103910,รับ
103911,การ
103912,โจมตี


In [ ]:
Data3 = Data3.replace('','_')

In [ ]:
Data3

,0
0,อย่างไรก็ตาม
1,เครื่อง
2,บิน
3,แอร์บัส
4,_
...,...
103909,พร้อม
103910,รับ
103911,การ
103912,โจมตี


In [ ]:
m=len(Data3)//256

In [ ]:
m

405

In [ ]:
emb_list3=[]
for i in tqdm(range(m+1)):
  if i==m:
    embeddings = model_emb.encode(list(Data3.iloc[256*i:len(Data3),0]))
  else:
    embeddings = model_emb.encode(list(Data3.iloc[256*i:(256*(i+1)),0]))
  emb_list3=emb_list3+list(embeddings)

In [ ]:
len(emb_list3)

103914

In [ ]:
len(emb_list3)

103680

In [ ]:
new_emb_list3 = []
for i in tqdm(range(0,len(emb_list3),17)):
  if i>len(emb_list3)-17:
    temp = emb_list3[i:len(emb_list3)]
    temp0 = [0]*768
    for j in range(17-(len(emb_list3)-i)):
      temp.append(temp0)
    new_emb_list3.append(temp)
  else:
    new_emb_list3.append(emb_list3[i:i+17])

100%|██████████| 6113/6113 [00:00<00:00, 451039.31it/s]


In [ ]:
emb_list3 = np.asarray(new_emb_list3,dtype='float32')

In [ ]:
emb_list3.shape

(6113, 17, 768)

In [ ]:
solution_t = model.predict(emb_list3)

In [ ]:
solution_t[0][-1]

array([1, 7, 2, 1, 0, 1, 9, 2, 5, 2, 6, 2, 7, 0, 0, 0, 0], dtype=int32)

In [ ]:
solution_arr[-10:]

[array([ 0.41186005,  0.36670464, -0.06030459, -0.45354006,  0.6354215 ,
        -0.03617806, -0.29955524, -1.2511351 , -0.25968236,  0.5162085 ,
         0.5842173 , -0.23596732,  1.285335  , -0.49517563,  0.05448327,
        -0.06756614, -0.20728898], dtype=float32),
 array([ 0.10158019,  0.6095236 ,  0.05346845,  0.6387213 , -0.27267587,
         0.29860535, -0.09092121, -0.88962716, -0.7456176 ,  0.3075279 ,
        -0.29413956, -0.04290921, -0.70123124, -0.136271  , -2.106405  ,
        -0.45421684,  0.04054083], dtype=float32),
 array([-0.25548658,  0.9294393 ,  0.87038183, -0.10876553, -0.48384377,
        -1.0232762 , -1.0861198 ,  0.26836985, -0.14353186, -0.19304301,
        -0.36569595, -1.2143192 , -1.5639814 ,  0.01073818, -1.1165761 ,
        -0.42062694, -0.2373863 ], dtype=float32),
 array([-5.5172962e-01,  5.7764786e-01, -3.8541418e-01, -9.4105446e-01,
         3.3763468e-01, -5.0132623e-04, -6.5321475e-01,  2.7053863e-01,
        -1.3531423e-01, -1.0069195e+00, -1.772

In [ ]:
solution_arr = [item for sublist in solution_t[0] for item in sublist]

In [ ]:
len(solution_arr)

103921

In [ ]:
result_t=[]
for i in solution_arr:
  result_t.append(inverselabel(Pos_unique,i))

In [ ]:
result_t=result_t[0:103914]

In [ ]:
Data3["result"]=result_t

In [ ]:
Data3=Data3.drop(103913,axis=0)

In [ ]:
Id = []
for i in range(1,len(Data3)+1):
  Id.append(i)

In [ ]:
len(Id)

103913

In [ ]:
solution1 = dict()
for i in range(len(Id)):
  solution1[Id[i]]=Data3["result"][i]

In [ ]:
solution1

{1: 'CC',
 2: 'NN',
 3: 'VV',
 4: 'NN',
 5: 'PU',
 6: 'NN',
 7: 'PU',
 8: 'AX',
 9: 'VV',
 10: 'CC',
 11: 'NN',
 12: 'PS',
 13: 'NN',
 14: 'NN',
 15: 'PS',
 16: 'NN',
 17: 'PU',
 18: 'NU',
 19: 'PU',
 20: 'CL',
 21: 'CC',
 22: 'VV',
 23: 'NN',
 24: 'VV',
 25: 'AJ',
 26: 'NU',
 27: 'CL',
 28: 'PU',
 29: 'NN',
 30: 'VV',
 31: 'VV',
 32: 'NN',
 33: 'VV',
 34: 'NN',
 35: 'VV',
 36: 'NN',
 37: 'NU',
 38: 'NN',
 39: 'PU',
 40: 'NU',
 41: 'PU',
 42: 'NN',
 43: 'PU',
 44: 'NU',
 45: 'NU',
 46: 'CL',
 47: 'PU',
 48: 'NN',
 49: 'NN',
 50: 'VV',
 51: 'NN',
 52: 'VV',
 53: 'NN',
 54: 'NN',
 55: 'VV',
 56: 'VV',
 57: 'NN',
 58: 'NU',
 59: 'NN',
 60: 'PS',
 61: 'NU',
 62: 'CL',
 63: 'CL',
 64: 'VV',
 65: 'NN',
 66: 'PS',
 67: 'PU',
 68: 'NU',
 69: 'PS',
 70: 'NN',
 71: 'VV',
 72: 'NN',
 73: 'NU',
 74: 'CL',
 75: 'VV',
 76: 'NN',
 77: 'NU',
 78: 'VV',
 79: 'VV',
 80: 'CC',
 81: 'VV',
 82: 'NN',
 83: 'CC',
 84: 'NN',
 85: 'NN',
 86: 'AJ',
 87: 'PU',
 88: 'NN',
 89: 'PU',
 90: 'VV',
 91: 'NN',
 92: 'VV

In [ ]:
#Import CSV
df = pd.read_csv('/content/super-ai-engineer-2021-nlp-pos-tagging/pos_sample_submission.csv')

#Updated 
df['Predicted'] = df[["Id","Predicted"]].apply(lambda x : solution1[x["Id"]] if x["Id"] in solution1.keys() else x["Predicted"],axis=1)  

#check not null
df["Predicted"].notnull().sum()

103913

In [ ]:
df.to_csv("/content/drive/MyDrive/POSTAG/submission.csv")

In [ ]:
df

,Id,Predicted
0,1,CC
1,2,NN
2,3,VV
3,4,NN
4,5,PU
...,...,...
103908,103909,VV
103909,103910,CC
103910,103911,VV
103911,103912,FX
